In [1]:
import LemurBounce2
l = LemurBounce2.LemurBounce2()

def saveWorld(key):
    l.sendOSCMessage('/saveWorld', 1, key)

def loadWorld(key):
    l.sendOSCMessage('/loadWorld', 1, key)

In [50]:
l.sendOSCMessage('/loadWorld', 1, '1')

/collision
[18.0, 1.0, 'start']


How can we have "musical" interactions with this state-replay mechanism?
- keep a running buffer of saved state every 16th or 32nd note. you can take "snapshots" of this running buffer. You can treat these snapshots as "notes" and resequence (like with tidalcycles) or reshuffle them. Experiment with using this technique for longer intervals. This is mainly to impose structure in worlds that are very chaotic. 
- In worlds that are less chaotic, and where user interaction changes some "parts" of the sound but also leaves most of the "parts" unchanged, you can get away with less frequent, more strategic changes. 
- goal - define worlds where the above type of "interaction by parts" is possible
- save/reload only subsets of the bodies, transform their positions and momentums. this would be especially useful in the "infinte non collision" world. 
- map movements/collions to "higher level" musical events, and deal with less objects in the world. 
- In general, we want to be able to modify the world with intentionality - meaning have a sense of what will be changed/preserved musically as the result of an interaction 